In [15]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#import API key
from config import g_key

#access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
#import cities.csv file into a dataframe
cities_df = pd.read_csv("cities.csv")

#get rid of 'City_ID' column
cities_df = cities_df[["City", "Cloudiness", "Country", 
                       "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Mumford,73,GH,1579838828,94,5.26,-0.76,76.96,3.36
3,Omboue,16,GA,1579838830,95,-1.57,9.26,75.83,0.36
4,Jamestown,90,US,1579838791,43,42.10,-79.24,39.00,5.82
5,Takoradze,2,GH,1579838832,93,4.88,-1.76,77.79,2.55
6,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51
7,Winneba,20,GH,1579838834,88,5.35,-0.62,82.40,4.70
8,Teshie,20,GH,1579838835,88,5.58,-0.11,82.40,4.70


In [3]:
#create heatmap based on humidity

#store latitude and longitude into the variable 'coordinates'
coordinates = cities_df[["Lat", "Lng"]]

#convert humidity to float; all data read from a csv is read as a string
humidity = cities_df["Humidity"].astype(float)

In [4]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

#add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Mumford,73,GH,1579838828,94,5.26,-0.76,76.96,3.36
3,Omboue,16,GA,1579838830,95,-1.57,9.26,75.83,0.36
4,Jamestown,90,US,1579838791,43,42.10,-79.24,39.00,5.82
5,Takoradze,2,GH,1579838832,93,4.88,-1.76,77.79,2.55
6,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51
7,Winneba,20,GH,1579838834,88,5.35,-0.62,82.40,4.70
8,Teshie,20,GH,1579838835,88,5.58,-0.11,82.40,4.70


In [12]:
specs_df = cities_df.loc[(cities_df["Max Temp"] > 40) &
                        (cities_df["Humidity"] < 93) & 
                        (cities_df["Wind Speed"] < 3) | 
                        (cities_df["Wind Speed"] > 8), :].reset_index()
specs_df = specs_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
specs_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51


In [13]:
nan_df = specs_df[specs_df.isna().any(axis=1)]
nan_df

hotel_df = specs_df.dropna()
hotel_df

#try this on the larger dataframe
#cities_nan = cities_df[cities_df.isna().any(axis=1)]
#cities_nan
#if there are nan values, then save by coding cities_df = cities_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51


In [14]:
#set up additional columns to hold information
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Port-Gentil,85,GA,1579838768,78,-0.72,8.78,83.10,8.57,
1,Anloga,20,GH,1579838827,88,5.79,0.90,80.60,2.24,
2,Gamba,96,GA,1579838833,85,-2.65,10.00,78.55,0.51,


In [26]:
#testing
#find a hotel near -.72, 8.78
target_search = "Hotel"
coordinates = "-0.72,8.78"
radius = 5000

# redefine params
params = {
    "location": coordinates,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Testing: Find a hotel near Port-Gentil, coordinates already given.")

# Run request
hotel_request = requests.get(base_url, params).json()

# Print the JSON (pretty printed)
print(json.dumps(hotel_request, indent=4, sort_keys=True))

# Print the name and address of the first hotel that appears (hotel name, city)
#print(hotel_request["results"][0]["name"])
#print(hotel_request["results"][0]["vicinity"])

Testing: Find a hotel near Port-Gentil, coordinates already given.
{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAKqmkQ6AFg5MIrT4wD6Em-1Sym0Js32it4ALcm7p59VnxssEhb61R-cqGa0on41xSjTHQmNMEaP3b000fMUV6LddKPpiSAH0PT9zza2NAJymMCnDscc6fN5OdVQp8P94bqWx-fen8X1zdtrDVJHbD-PNjvuON-T07kr-dUyBx0kaO27N17FSBYLfVfaqb3YpUIg2lXbxuLGEgVLF6s9kzLJzYRW1WEt3kG1vW7gXnOKhABcdDvwVZxgNXGt5EUHVSN8ML_-rPkDz6lUX0VRUMOvL-ZeEJT8nNXYEjZpOIT4FdWTVg6e1-q5T6jo17hNe2F070IaWhbJhqVTd_toO-5VkUy6r6dPCK0-at5UTYMNMJECRma4EeqOAPRbTXv237hIQdz9v4ZHacL58W1cJVwIULBoU_9vdyn1wjhel9ghSSf4udW9ZXAg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -0.694422,
                    "lng": 8.7651822
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.6929901201072778,
                        "lng": 8.766922629892722
                    },
                    "southwest": {
                        "la